In [1]:
import pandas as pd
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt

f = pd.read_csv('Week_8/LRflutterx4_h2.txt', sep = ' ', header=None)
f2 = pd.read_csv('ConnectFour-master/Spikerbox/Week_10/LRflutter_a.txt', sep = ' ', header=None)
f3 = pd.read_csv('ConnectFour-master/Spikerbox/Week_10/LRflutter_a2.txt', sep = ' ', header=None)
f4 = pd.read_csv('ConnectFour-master/Spikerbox/Week_10/LRflutter_h1.txt', sep = ' ', header=None)
f5 = pd.read_csv('ConnectFour-master/Spikerbox/Week_10/LRflutter_s3.txt', sep = ' ', header=None)
f6 = pd.read_csv('ConnectFour-master/Spikerbox/Week_10/LRflutter_s4.txt', sep = ' ', header=None)
f7 = pd.read_csv('ConnectFour-master/Spikerbox/Week_10/LRflutter_s5.txt', sep = ' ', header=None)
f8 = pd.read_csv('ConnectFour-master/Spikerbox/Week_10/LRflutter_s6.txt', sep = ' ', header=None)

x = 200

data = f.iloc[:,1]
data = data.to_numpy()
data = np.flip(data)
a1 = []
i = 0
while i <len(data):     #Taking rolling average for x points
    a1.append(np.mean(data[i:i+x]))
    i += x

data2 = f2.iloc[:,0]
data2 = data2.to_numpy()
data2 = np.flip(data2)
data2 = data2[180000:]
a2 = []
i = 0
while i <len(data2):     #Taking rolling average for x points
    a2.append(np.mean(data2[i:i+x]))
    i += x

data3 = f3.iloc[:,0]
data3 = data3.to_numpy()
data3 = np.flip(data3)
a3 = []
i = 0
while i <len(data3):     #Taking rolling average for x points
    a3.append(np.mean(data3[i:i+x]))
    i += x

data4 = f4.iloc[:,0]
data4 = data4.to_numpy()
data4 = np.flip(data4)
a4 = []
i = 0
while i <len(data4):     #Taking rolling average for x points
    a4.append(np.mean(data4[i:i+x]))
    i += x

data5 = f5.iloc[:,0]
data5 = data5.to_numpy()
data5 = np.flip(data5)
a5 = []
i = 0
while i <len(data5):     #Taking rolling average for x points
    a5.append(np.mean(data5[i:i+x]))
    i += x

data6 = f6.iloc[:,0] #good
data6 = data6.to_numpy()
data6 = np.flip(data6)
a6 = []
i = 0
while i <len(data6):     #Taking rolling average for x points
    a6.append(np.mean(data6[i:i+x]))
    i += x

data7 = f7.iloc[:,0] 
data7 = data7.to_numpy()
data7 = np.flip(data7)
a7 = []
i = 0
while i <len(data7):     #Taking rolling average for x points
    a7.append(np.mean(data7[i:i+x]))
    i += x

data8 = f8.iloc[:,0]
data8 = data8.to_numpy()
data8 = np.flip(data8)
a8 = []
i = 0
while i <len(data8):     #Taking rolling average for x points
    a8.append(np.mean(data8[i:i+x]))
    i += x
a8 = a8[1100:]

In [8]:
def Detection(seq, std_thresh, diff_thresh, prom_threshold):
    std = np.std(seq)
    diff = np.max(seq) - np.min(seq)
    peaks = len(scipy.signal.find_peaks(seq, prominence=prom_threshold)[0])
    maxval = np.argmax(seq)
    minval = np.argmin(seq)
    if peaks > 2:
        return 'fl'
    elif std > std_thresh and diff > diff_thresh:
        if maxval > minval:
            return 'L'
        else:
            return 'R'
    else:
        return 'NA'

In [11]:
l = len(a1)
data = a1
waiting = False
b = 0
calibrating = True
start = True
cal_count = 0
nas = 0
last_seq = np.empty(0)
max_cstd = 0
max_cdiff = 0
max_ch = 0
current_detection = 0
count = 0
best_diff = 0
cbest_diff = 0
ccount = 0
cstds = np.empty(0)
cdiffs = np.empty(0)

while b < l:
    if l-b < 50:
        data_temp = data[b:(b-1)]
    else:
        data_temp = data[b:(b+50)]
    
    data_temp = np.array(data_temp)
    combined = np.concatenate((last_seq, data_temp), axis = None)

    if calibrating and b > 0:
        if start:
            arrs = np.split(data_temp, 5)
            means = np.empty(0)
            diffs = np.empty(0)
            for k in arrs:
                means = np.append(means, np.mean(k))
                diffs = np.append(diffs, (np.max(k) - np.min(k)))
            base_h = np.median(means)
            base_std = np.std(data_temp)
            if base_std > 5:
                base_std = 5
            base_diff = np.median(diffs)
            start = False
        else:
            c = 0
            movement = len(combined) - 50
            while movement - c > 0:
                interval = combined[c:(c+50)]
                cstd = np.std(interval)
                cdiff = np.max(interval) - np.min(interval)
                ch = np.max(interval)
                cpeaks = len(scipy.signal.find_peaks(interval, prominence=10)[0])
                if cpeaks >= 3:
                    calibrating = False
                    cpeaks = 0
                    cstds = np.flip(np.sort(cstds))
                    cdiffs = np.flip(np.sort(cdiffs))
                    fstd = cstd
                    fdiff = cdiff
                    fch = ch
                    
                    max_std = cstds[0]
                    if len(cstds) > 3:
                        low_std = cstds[2]
                    elif len(cstds) == 1:
                        low_std = max_std
                    else:
                        low_std = cstds[1]
                    std_threshold = low_std/2
                    
                    max_diff = cdiffs[0]
                    if len(cdiffs) > 3:
                        low_diff = cdiffs[2]
                    elif len(cdiffs) == 1:
                        low_diff = max_diff
                    else:
                        low_diff = cdiffs[1]
                    diff_threshold = low_diff/2

                    prom_threshold = 10
                    break
                    
                elif cstd > 2*base_std and ccount > 4:
                    if cdiff >= cbest_diff:
                        cbest_diff = cdiff
                        cbest = interval
                    elif (cbest_diff - cdiff) > 10:
                        cstds = np.append(cstds, np.std(cbest)) 
                        cdiffs = np.append(cdiffs, cbest_diff)
                        cbest = 0
                        cbest_diff = 0
                        ccount = 0
                else:
                    ccount += 1
                
                c += 5

    elif b > 0:
        d = 0
        if len(combined) > 50:
            movement = len(combined) - 50
            while movement - d > 0:
                interval = combined[d:(d+50)]
                predicted = Detection(interval, std_threshold, diff_threshold, prom_threshold)
                ran = np.max(interval) - np.min(interval)
                sdi = np.std(interval)

                if predicted == 'NA':
                    count += 1

                elif predicted == 'fl' and count > 5:
                    count = 0
                    best = 0
                    best_diff = 0
                    print(predicted)
                    # predicted is input to game here

                elif (sdi > std_threshold or ran > diff_threshold) and count > 5:
                    if ran >= best_diff:
                        best_diff = ran
                        best = interval
                    elif (best_diff - ran) > 10:
                        actual = Detection(best, std_threshold, diff_threshold, prom_threshold)
                        print(actual)
                        # actual is input to game here
                        best = 0
                        best_diff = 0
                        count = 0
                previous = predicted
                d += 5
    last_seq = data_temp
    b += 50

L
R
fl
L
R
fl
L
R
fl
